<a ><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Logo_INSA_Lyon_%282014%29.svg/langfr-2560px-Logo_INSA_Lyon_%282014%29.svg.png"  width="200" align="left"> </a>
<div style="text-align: right"> <h3><span style="color:gray"> Projet de recherche </span> </h3> </div>

<br>
<br>
<br>


<h1><center>Data Analysis</center></h1>
<h2><center> <span style="font-weight:normal"><font color='#e42618'> Creating Targets and Features based on analyst reports and market data</font>  </span></center></h2>


<h3><center><font color='gray'>JONAS GOTTAL</font></center></h3>





<h4>Project scope</h4>

- Obtaining financial research containing both a report in natural language and a quantifiable prediction on the underlying asset
- Building a predictive models that e.g., accurately detect semantic causality
- Evaluation whether there is a correlation between causal formulations and a higher accuracy from an analyst
<br>

---
---

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="text-decoration:none; margin-top: 30px; background-color:#F2F2F2; border-color:#720006">
    <span style="color:#720006">
    <ol>
        <li><a href="#1"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Design of meaningful Targets</span> </a></li>
       <ol>
       <li><a href="#2"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Binary Accuracy</span> </a></li>
       <li><a href="#3"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Change of underlying trend</span> </a></li>
        <li><a href="#4"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Accuracy of predicted change of trend</span> </a></li>
       </ol>
       <li><a href="#5"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Exploratory Analysis: detection of Features</span> </a></li> 
       <ol>
       <li><a href="#6"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Absence and presence of certain words</span> </a></li>
       <li><a href="#7"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Active, Passive and other semantic markers</span> </a></li>
        <li><a href="#8"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Large Language Models: Sentiment, Causality and more</span> </a></li>
       </ol>
       <li><a href="#9"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Preliminary Conclusion</span> </a></li>
    </ol>
    </span>
</div>

#### Requirements
- ```Python 3.9.18``` (conda env)
- ```pip freeze > requirements.txt```
- ```conda env export > environment.yml```

# Design of meaningful Targets <a id="1"></a>

### Load Data

In [1]:
import json
import pandas as pd
import numpy as np

# get date as datetime
from datetime import datetime

# Load the data from the JSON file
with open('Data/30d_data/all_data.json') as f:
    all_data = json.load(f)

# Convert each item in the dictionary back into a DataFrame
all_data_df = {key: pd.read_json(value, orient='columns') for key, value in all_data.items()}

df = pd.read_csv('Data/final_data.csv', index_col=0)


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/658643737.py:13: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_data_df = {key: pd.read_json(value, orient='columns') for key, value in all_data.items()}
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/658643737.py:13: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_data_df = {key: pd.read_json(value, orient='columns') for key, value in all_data.items()}
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/658643737.py:13: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_data_df = {key: pd.read_json(value, orient='colum

In [2]:
#test 
all_data_df['VMUK.L']

,Open,High,Low,Close,Adj Close,Volume
2023-10-23,152.649994,154.699997,151.149994,154.399994,154.399994,1511370
2023-10-24,150.699997,157.050003,149.149994,151.350006,151.350006,2441229
2023-10-25,150.800003,153.600006,146.949997,149.500000,149.500000,2007126
2023-10-26,146.500000,149.699997,146.050003,149.600006,149.600006,2108166
2023-10-27,149.000000,149.000000,145.300003,146.250000,146.250000,2679563
2023-10-30,144.850006,149.649994,143.250000,148.899994,148.899994,5908708
2023-10-31,144.699997,151.000000,144.699997,148.949997,148.949997,4564782
2023-11-01,150.000000,152.940994,148.399994,152.199997,152.199997,1790671
2023-11-02,155.949997,157.199997,153.550003,156.449997,156.449997,3693647
2023-11-03,157.800003,161.175003,156.550003,158.850006,158.850006,1515255


### Get moving average before and after

In [3]:
# for adjusted close in all_data_df[symbol], calculate the moving average from the start to the date (df["PriceDate"][0]) and from that date to the end
# and save in df["30dAvgPrice"][0]

df["30dAvgPriceBefore"] = np.nan
df["30dAvgPriceAfter"] = np.nan
for i in range(len(df)):
    symbol = df["FinalTickerSymbol"][i]
    date = df["PriceDate"][i]
    # turn date into datetime
    date = datetime.strptime(date, '%d/%m/%Y')
    date = date.strftime('%Y-%m-%d')
    try:
        df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
        df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
    except:
        pass
    # print progress
    print(f"{i+1}/{len(df)}")

/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

1/1626
2/1626
3/1626
4/1626
5/1626
6/1626
7/1626
8/1626
9/1626
10/1626
11/1626
12/1626
13/1626
14/1626
15/1626
16/1626
17/1626
18/1626
19/1626
20/1626
21/1626
22/1626


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

23/1626
24/1626
25/1626
26/1626
27/1626
28/1626
29/1626
30/1626
31/1626
32/1626
33/1626
34/1626
35/1626
36/1626
37/1626
38/1626
39/1626
40/1626
41/1626
42/1626
43/1626
44/1626
45/1626
46/1626
47/1626
48/1626
49/1626
50/1626
51/1626
52/1626
53/1626
54/1626
55/1626
56/1626
57/1626
58/1626
59/1626
60/1626
61/1626
62/1626
63/1626
64/1626
65/1626


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

...

/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

1315/1626
1316/1626
1317/1626
1318/1626
1319/1626
1320/1626
1321/1626
1322/1626
1323/1626
1324/1626
1325/1626
1326/1626
1327/1626
1328/1626
1329/1626
1330/1626
1331/1626
1332/1626
1333/1626
1334/1626
1335/1626
1336/1626
1337/1626
1338/1626
1339/1626
1340/1626
1341/1626
1342/1626
1343/1626
1344/1626
1345/1626
1346/1626
1347/1626
1348/1626
1349/1626
1350/1626
1351/1626
1352/1626
1353/1626
1354/1626
1355/1626


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

1356/1626
1357/1626
1358/1626
1359/1626
1360/1626
1361/1626
1362/1626
1363/1626
1364/1626
1365/1626
1366/1626
1367/1626
1368/1626
1369/1626
1370/1626
1371/1626
1372/1626
1373/1626
1374/1626
1375/1626
1376/1626
1377/1626
1378/1626
1379/1626
1380/1626
1381/1626
1382/1626
1383/1626
1384/1626
1385/1626
1386/1626
1387/1626
1388/1626
1389/1626
1390/1626
1391/1626
1392/1626
1393/1626
1394/1626
1395/1626
1396/1626
1397/1626
1398/1626
1399/1626
1400/1626
1401/1626
1402/1626
1403/1626
1404/1626
1405/1626
1406/1626
1407/1626
1408/1626
1409/1626
1410/1626
1411/1626
1412/1626
1413/1626
1414/1626
1415/1626
1416/1626
1417/1626
1418/1626
1419/1626
1420/1626
1421/1626
1422/1626
1423/1626
1424/1626
1425/1626
1426/1626
1427/1626
1428/1626
1429/1626
1430/1626
1431/1626
1432/1626
1433/1626
1434/1626
1435/1626
1436/1626
1437/1626
1438/1626
1439/1626
1440/1626
1441/1626
1442/1626
1443/1626
1444/1626
1445/1626
1446/1626
1447/1626
1448/1626
1449/1626
1450/1626
1451/1626
1452/1626
1453/1626
1454/1626
1455/1626


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["30dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_32385/840846061.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [4]:
df

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice,IsClose,30dAvgPriceBefore,30dAvgPriceAfter
0,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VMUK,4,24/11/2023,Nathan Zaia,146.75,GBP,23/11/2023,...,VMUK.L,Financial Services,1.932661e+09,LSE,45564.0,GBp,157.000000,True,157.710417,154.075000
1,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VUK,4,24/11/2023,Nathan Zaia,2.78,AUD,24/11/2023,...,VUK.AX,Financial Services,3.709752e+09,ASX,2057263.0,AUD,2.960000,True,2.994167,2.940476
2,24/11/2023,Qube's Good Momentum Likely to Carry Into Fisc...,Qube Holdings Ltd,QUB,4,24/11/2023,Adrian Atkins,2.93,AUD,24/11/2023,...,QUB.AX,Industrials,5.229462e+09,ASX,2520549.0,AUD,2.930000,True,2.840000,3.059524
3,24/11/2023,Stabilization of Political Risk in Spain Is Po...,Endesa SA,ELE,4,23/11/2023,Tancrede Fulop,18.84,EUR,24/11/2023,...,0N9G.IL,Utilities,2.130355e+10,IOB,5925.0,EUR,18.764191,True,18.198966,18.992456
4,24/11/2023,Sage Has Reached an Inflection Point in Its Cl...,Sage Group (The) PLC,SGE,1,23/11/2023,Rob Hales,1130.50,GBP,23/11/2023,...,SGE.L,Technology,1.173311e+10,LSE,262362.0,GBp,1130.000000,True,980.521973,1142.985268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,25/11/2023,Minimal Changes To Our Fair Value After Fiscal...,The Toronto-Dominion Bank,TD,4,NaN,Eric Compton,83.15,CAD,23/11/2023,...,TD.TO,Financial Services,1.466779e+11,TOR,4963474.0,CAD,83.360001,True,80.830834,82.840454
1622,25/11/2023,Xiabuxiabu's Value-Oriented Model Should Fare ...,Xiabuxiabu Catering Management (China) Hldgs C...,00520,4,28/08/2023,Ivan Su,3.25,HKD,24/11/2023,...,0520.HK,NaN,NaN,NaN,NaN,NaN,3.290000,True,3.062500,2.674762
1623,25/11/2023,NextDC's Australia Wide Data Center Network Ex...,Nextdc Ltd,NXT,3,28/08/2023,Dan Baker,12.92,AUD,24/11/2023,...,NXT.AX,Technology,6.918949e+09,ASX,1956904.0,AUD,13.070000,True,12.505417,13.200952
1624,25/11/2023,EVT on Track To Return to Prepandemic Earnings,EVT Ltd,EVT,4,28/08/2023,Brian Han,10.82,AUD,24/11/2023,...,EVT.AX,Communication Services,1.720115e+09,ASX,52356.0,AUD,10.830000,True,10.715417,11.481429


### Stock Trend before and after release of report

We check whether the stock is growing ```(1)```, constant ```(0)``` or falling ```(-1)``` for two points of time: at release of report and 30d after. We also design a binary variable in the same way whether the analyst estimates the stock to grow, fall or move sideways. This enables us to design our desired Targets and also obtain some basic metrics:
- Trend of predictions
- Accuracy of predictions 
- And whether the accuracy is dependent on the prediction itself -> sub categories

